In [ ]:
import json

with open('landfills_openstreetmap.geojson') as f:
    data = json.load(f)

In [ ]:
len(data)

In [ ]:
data.keys()

In [ ]:
data['type'], data['generator'], data['copyright'], data['timestamp']

In [ ]:
len(data['features'])

In [ ]:
data['features'][0]

In [ ]:
# These are the main includes used through the notebook
import datetime
import geojson
import numpy as np                   # numeric linear algebra
import matplotlib.pyplot as plt      # plotting
import rasterio       # read/write geotiffs
import tsd            # time series downloader for sentinel and landsat
import utils          # IO and coordinate system conversion tools
import vistools       # display tools
import folium
import folium.plugins

In [ ]:
geojsonstring = str(data['features'][0])

In [ ]:
print(geojsonstring)

In [ ]:
import json

# Replace single quotes with double quotes in the JSON string
geojsonstring = geojsonstring.replace("'", '"')

# interpret the string as an object
geojsonstruct = geojson.loads(geojsonstring)

# this extracts the geometry field
aoi = utils.find_key_in_geojson(geojsonstruct,'geometry')

print("These are the coordinates of the selected polygon in GeoJSON format:")
print(geojson.dumps(aoi, indent=2))

In [ ]:
m = vistools.foliummap()

# we can draw the aoi
folium.GeoJson(aoi).add_to(m)
# and center the map at the aoi center
m.location   = np.mean(aoi['coordinates'][0][:4], axis=0).tolist()[::-1]

display(m)

In [ ]:
aoi = utils.find_key_in_geojson(geojsonstruct,'geometry')

In [ ]:
start_date = datetime.datetime(2023, 7, 1)
end_date = datetime.datetime(2023, 12, 31)

# run the query
image_catalog = tsd.get_sentinel2.search(aoi, start_date=start_date, end_date=end_date)

if len(image_catalog) == 0:
    print("Maybe your AOI selection is too large")

import pandas as pd
pd.DataFrame([dict(i) for i in image_catalog])

In [ ]:
## Let's select the last entry from the catalog
entry = image_catalog[-1]

# and display its thumbnail
vistools.display_image(entry['thumbnail'])   #  for displaying an image in the notebook
print('At any moment, about 60 percent of the earth is covered by clouds.')

In [ ]:
def query_clear_sky(aoi, satellite='Sentinel-2', max_cloud_cover=30,
                    start_date=None, end_date=None):
    '''
    queries the devseed database for the aoi
    returns a filtered catalog with images with cloud
    cover of at most max_cloud_cover
    '''
    # run a search
    if satellite == 'Sentinel-2':
        res = tsd.get_sentinel2.search(aoi, start_date, end_date)
    elif satellite == 'Landsat-8':
        res = tsd.get_landsat.search(aoi, start_date, end_date)


    ###### Insert your solution code here ######
    res2 = []
    for image in res:
      if image.cloud_cover <= max_cloud_cover:
          res2.append(image)

    return res2

In [ ]:
res = query_clear_sky(aoi, satellite='Sentinel-2', max_cloud_cover=20,
                      start_date=start_date,
                      end_date=end_date)

## display the images in a gallery
g, l = [], []
for p in res:
    l.append("Imaging date: {}, cloud cover: {}".format(p.date.isoformat(), p.cloud_cover))
    g.append(p.thumbnail)

vistools.display_gallery(g, l)

In [ ]:
def get_crop_from_aoi(output_path, aoi, catalog_entry, band):
    """
    Crop and download an AOI from a georeferenced image.
    Args:
        output_path (string): path to the output GeoTIFF file
        aoi (geojson.Polygon): area of interest defined by a polygon in longitude, latitude coordinates
        catalog_entry (tsd.s2_metadata_parser.Sentinel2Image): metadata object
        band (str): desired band, e.g. 'B04' for Sentinel-2 or 'B8' for Landsat-8
    """
    metadata = catalog_entry
    if not metadata.urls['aws']:
        metadata.build_s3_links()
    inpath = metadata.urls['aws'][band]

    utm_zone = metadata.get("utm_zone")
    lat_band = metadata.get("lat_band")
    epsg = tsd.utils.utm_to_epsg_code(utm_zone, lat_band) if utm_zone else None

    ulx, uly, lrx, lry, epsg = tsd.utils.utm_bbx(aoi, epsg=epsg, r=60)
    tsd.utils.rasterio_geo_crop(output_path, inpath, ulx, uly, lrx, lry, epsg=epsg)

In [ ]:
# # lets download the image
# output_path = 'output.tiff'
# get_crop_from_aoi(output_path, aoi, res[-1], 'B04')

# # and display it
# vistools.display_image(output_path)


In [ ]:
# Let's start downloading the green channel
my_satellite, band = 'Sentinel-2', 'B03'
fname = 'ex4_{}.tif'.format(band)
results = query_clear_sky(aoi, my_satellite, max_cloud_cover=30,
                          start_date=start_date, end_date=end_date)

In [ ]:
# This is where the downloading is done
get_crop_from_aoi(fname, aoi, results[0], band)

# To read the image and metadata useutils.readGTIFF andutils.readGTIFFmeta
im   = utils.readGTIFF(fname)
meta = utils.readGTIFFmeta(fname)

print('We can see some metadata...')
display(meta)
print('...and the image. However it has a high bit depth so it cannot be shown as it is')
vistools.display_image(im)
print('we can divide by 32 and see what happens')
vistools.display_image(im/32)
print('the result seems ok, but a bit dark')

In [ ]:
def simple_equalization_8bit(im, percentiles=5):
    ''' im is a numpy array
        returns a numpy array
    '''

    ###     <<< CODE HERE >>>
    low = np.percentile(im, percentiles)
    high = np.percentile(im, 100-percentiles)

    im = np.clip(im, low, high)
    im = (im - low) / (high - low)
    im = np.uint8(im * 255)

    return im

In [ ]:
basename='ex4_'
im = utils.readGTIFF(basename+'B03.tif')
vistools.display_image(
    simple_equalization_8bit(im, percentiles=2)
)

In [ ]:
#  Let's write a function that downloads the Sentinel-2 RGB bands
#  and creates an 8bit color image. For that we will apply the function
#  simple_equalization_8bit to each channel.

def get_sentinel2_color_8bit(basefilename, aoi, catalog_entry, percentiles=5):
    ''' basefilename to store the bands:  basename+BAND+'.tif'
        returns a numpy array of the RGB image (height, width, channels)
    '''
    bands = ['B02', 'B04', 'B03']    # SENTINEL2 R,G,B BANDS

    # this command downloads all the bands
    for b in bands:
        get_crop_from_aoi('{}_{}.tif'.format(basefilename, b), aoi, catalog_entry, b)

    # read, equalize, and stack all the channels
    out = []
    for b in bands:
        im = utils.readGTIFF('{}_{}.tif'.format(basefilename, b))
        im = simple_equalization_8bit(im, percentiles)
        out.append(im)

    # The transposition is necessary because the indexing
    # convention for color images is (height, width, channels)
    im = np.squeeze(out,axis=(3)).transpose(1,2,0)
    return im


#### Test the function by running
# pick the satellite then query the database
my_satellite = 'Sentinel-2'
basename = 'rgb'
res = query_clear_sky(aoi, my_satellite, max_cloud_cover=10, start_date=start_date, end_date=end_date)

# generate the RGB image
RGBout = get_sentinel2_color_8bit(basename, aoi, res[-1])

# Writes RGBout in 'rgb_RGB.tif' copying geolocation metadata from 'rgb_B03.tif',
# which has been written by    get_sentinel2_color_8bit
utils.writeGTIFF(RGBout, basename+'_RGB.tif', basename+'_B03.tif')

# display RGBout
vistools.display_imshow(RGBout, figsize=(10,10))